In [1]:
#default_exp node

In [2]:
#hide
#workaround to make relative imports inside notebook
import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

# Dependencies

In [3]:
#export
import d6tflow

import os
from pathlib import Path
from warnings import warn
from collections import defaultdict

from sklearn.base import BaseEstimator
from sklearn.exceptions import NotFittedError
from sklearn.preprocessing import FunctionTransformer

from scipy import sparse
import numpy as np

Loading postgres module without psycopg2 installed. Will crash at runtime if postgres functionality is used.


Welcome to d6tflow! For Q&A see https://github.com/d6t/d6tflow


# Input handling
> fucntionalities to handle inputs from tasks

In [4]:
#export
#TODO: define node name validaton rules
def _validate_name(name):
    '''
    function to validate node names.
    for now, any name is accepeted.
    '''
    return name

def _process_multiple_kwargs(kwargs_list):
    '''
    takes a list of kwargs and returns a kwargs dict with parameters as lists
    '''
    
    keys = []
    for kwargs in kwargs_list:
        keys += list(kwargs.keys())
    
    keys = list(set(keys))
    #TODO: define better strategy
    #dict will contain __EMPTY_KW_PARAM__ if parameter has not been passed
    dict_kwargs = {key:['__EMPTY_KW_PARAM__']*len(kwargs_list) for key in keys}
    
    for idx in range(len(kwargs_list)):
        kwargs = kwargs_list[idx]
        for key in kwargs:
            dict_kwargs[key][idx] = kwargs[key]
    
    return dict_kwargs

def unpack_inputs(inputs, arg_names = ['X']):
    '''
    unpacks result of Task.inputLoad(), according to defined convention for fit tasks
    returns tuple (X, kwargs)
    '''    
    if len(inputs) < 1:
        raise ValueError('No inputs to unpack')
    
    #checks if keys contains arg_names
    if (set(arg_names) & set(inputs.keys())) == set(arg_names):
        #id single input, return array only
        
        args = [inputs[name] for name in arg_names]        
        kwargs = {k:v for k,v in inputs.items() if not k in arg_names}
    
    else:
        #in d6tflow, when there are multiple inputs, values are passed as a dict with ascending numerical keys 
        args = [[] for _ in range(len(arg_names))]
        kwargs = []   
        
        for idx in inputs:            
            for argidx in range(len(arg_names)):                                
                args[argidx].append(inputs[idx][arg_names[argidx]])                
            
            kwargs.append({k:v for k,v in inputs[idx].items() if not k in arg_names})
    
        kwargs = _process_multiple_kwargs(kwargs)
        #return tuples
        args = [tuple(arg) for arg in args]
        kwargs = {k:tuple(v) for k,v in kwargs.items()}                              
    
    return (*args, kwargs)

def unpack_transform_inputs(inputs):
    '''
    unpacks result of Task.inputLoad(), according to defined convention for transform tasks
    returns tuple (estimator, X, kwargs)
    '''
    
    if len(inputs) < 1:
        raise ValueError('No inputs to unpack')
            
    #retrieves estimator (will always be dict since transform needs at least 
    #two inputs always: estimator (from fit) and input)
    estimator = inputs[0]
    #remove estimator from inputs
    del inputs[0]
    
    #unpack dict if len == 1 to avoid breaking unpack_inputs
    if len(inputs) == 1:
        inputs = list(inputs.values())[0]
    
    X, kwargs = unpack_inputs(inputs, arg_names = ['X'])
    
    return estimator, X, kwargs

# Task factories
> Factories to dinamically create d6tflow task classes

In [5]:
#export
#TODO: disable fit and transform method for singletons (using @property ro raise a erro when attr called), enable only in pipeline
def input_task_factory(task_name, **kwargs):
    '''
    task to dump any object as a pkl file
    '''    
    class DumpTask(d6tflow.tasks.TaskPickle):
        def run(self,):            
            self.save(kwargs)
            return
    
    DumpTask.__name__ = _validate_name(task_name)
    return DumpTask    

def fit_task_factory(task_name, estimator, dependencies = [], y_task = None):
    '''
    creates a task to fit estimator    
    '''
    assert isinstance(dependencies, (list, tuple)), f'dependencies should be tuple or list, not {type(dependencies)}'
        
    class FitTask(d6tflow.tasks.TaskPickle):
        
        def run(self,):                                                
            #runs pipeline only if oject does not exist already
            path = Path(self.output().path)
            
            if not path.exists():                
                inputs = self.inputLoad()
                #reset and rerun y_loader
                y_task().reset(confirm = False)
                y_task().run()

                y = y_task().outputLoad()
                y, *_ = unpack_inputs(y, arg_names = ['y'])

                X, kwargs = unpack_inputs(inputs, arg_names = ['X'])
                estimator.fit(X, y, **kwargs)            

                self.save(estimator)
            else:
                pass
            
            return
    
    #different syntax for @requires wrapper    
    if len(dependencies) > 0:        
        FitTask = d6tflow.requires(*dependencies)(FitTask)
    
    FitTask.__name__ = _validate_name(task_name)
    return FitTask

def transform_task_factory(task_name, fit_task, input_dependencies = [], transform_method = 'transform'):
    '''
    receives X and **kwargs from dependencies, performs and saves result from estimator.transform()
    '''
    assert isinstance(input_dependencies, (list, tuple)), f'dependencies should be tuple or list, not {type(input_dependencies)}'
    
    class TransformTask(d6tflow.tasks.TaskPickle):
        def run(self,):
            inputs = self.inputLoad()
            estimator, X, kwargs = unpack_transform_inputs(inputs)
            results = getattr(estimator, transform_method)(X, **kwargs)
            self.save({'X':results})
            return
    
    
    TransformTask = d6tflow.requires(fit_task, *input_dependencies)(TransformTask)
    
    TransformTask.__name__ = _validate_name(task_name)
    return TransformTask

# Nodes
> Node InputTransformer and NodeTransformer, classes to wrap sklearn estimators and compose final DAGEstimator

In [6]:
#export
def _validate_input_nodes(input_nodes, child_name):
    '''
    checks if input nodes are valid (NodeTransformer).
    If not NodeTransformer but valid BaseEstimator instance, wrapps BaseEstimator in NodeTransformer
    '''
        
    processed_nodes = []
    for node in input_nodes:
        if isinstance(node, NodeTransformer):
            processed_nodes.append(node)
        
        elif isinstance(node, BaseEstimator):
            node = NodeTransformer(node, 'Parent' + child_name)
            process_nodes.append()

## BaseNode

In [7]:
#export
class BaseNode(BaseEstimator):
    '''
    Base class for nodes
    overwrites sklearn base estimator __reppr__ and __str__
    since it takes too long to show in nested structures
    '''    
    
    def __repr__(self,):
        return self.name        
        

## InputTransformer

In [8]:
#export
class InputTransformer(BaseNode):
    
    def __init__(self, name = None):
                                        
        if name is None:
            self.name = str(id(self)) + '__input'
        else:
            self.name = _validate_name(name) + '__input'
        #input task placeholder, to access this task, fit_task and input_task aliases will be created, for consistency
        self.__input_task = None        
        self.__output_path = None
        return
    
    
    @property
    def input_task(self,):                
        
        if self.__input_task is None:
            raise NotFittedError(f'Input node {self.name} is not fitted, fit it prior to accessing its tasks')
        
        return self.__input_task
            
    @property
    def output_path(self,):
        return self.__output_path
    
    @property
    def fit_task(self,):
        return self.input_task
    
    @property
    def transform_task(self,):
        return self.input_task
    
    def _make_input_task(self, X, **kwargs):
        self.fit(X,**kwargs)
        return
    
    def _make_fit_task(self, X, **kwargs):
        '''does nothing'''
        return
    
    def _make_transform_task(self, **kwargs):
        '''does nothing'''
        return
            
    def fit(self, X, **kwargs):        
        '''
        creates input task. saves kwargs
        '''
        kwargs = {'X':X, **kwargs}
        self.__input_task = input_task_factory(self.name, **kwargs)
        self.__output_path = self.__input_task().output().path
        return self
    
    def transform(self, X = None, **kwargs):        
        self.input_task.reset(confirm = False)
        self.input_task.run()
        result = self.input_task.outputLoad()        
        return result
    

## NodeTransformer

In [9]:
#export
class NodeTransformer(BaseNode):
    
    def __init__(self, estimator, input_nodes = [], name = None, transform_method = 'transform'):
        
        self.estimator = estimator
        
        if not isinstance(input_nodes, (list, tuple)) and (len(input_nodes) > 0):
            raise TypeError(f'input_nodes should be a list or tuple with length greater than 0, got {input_nodes}')
        self.input_nodes = input_nodes
        
        if name is None:
            self.name = 'Task' + str(id(self))
        else:
            self.name = _validate_name(name)                                        
        
        self.transform_method = transform_method
        
        #placeholder for __y_loader
        self.__y_loader = None
        self.__fit_task = None
        self.__transform_task = None
        self.__estimator_path = None
        self.__output_path = None
        return
    
            
    def _set_y_loader(self, y_loader):
        '''
        sets self.__y_loader, needed during fit
        '''
        self.__y_loader = y_loader
        return
                        
    @property
    def y_loader(self,):
        
        if self.__y_loader is None:
            raise NotFittedError(f'Should run {self.name}._set_y_loader prior to accessing. Do it throguh _set_y_loader method')
        
        return self.__y_loader
    
    @property
    def fit_task(self,):        
        
        if self.__fit_task is None:
            raise NotFittedError(f'Should run {self.name}._make_fit_task prior to accessing it')
        
        return self.__fit_task
        
    @property
    def transform_task(self):        
        '''
        transform task property.
        
        '''
        
        if self.__transform_task is None:                                    
            raise NotFittedError(f'Should run {self.name}._make_transform_task() prior to accessing it')
        
        return self.__transform_task
    
    @property
    def output_path(self,):
        return self.__output_path
    
    @property
    def estimator_path(self,):
        return self.__estimator_path
    
    def _make_transform_task(self, transform_method = None):
        '''
        creates mangled (private) __transform_task attribute
        '''
        
        if transform_method is None:
            transform_method = self.transform_method
        else:
            transform_method = transform_method
            
        
        
        #create transform_task in children if needed
        input_tasks = []
        for children in self.input_nodes:
            
            children._make_transform_task()
            input_tasks.append(children.transform_task)
            
        #create fit_task in parent if needed
        try:
            self.__transform_task = transform_task_factory(
                self.name + f'__{transform_method}', self.fit_task, input_tasks, transform_method = transform_method
            )
        except NotFittedError:            
            self._make_fit_task()
            self.__transform_task = transform_task_factory(
                self.name + f'__{transform_method}', self.fit_task, input_tasks, transform_method = transform_method
            )
                
        
        self.__output_path = self.transform_task().output().path
        return
    
    def _make_fit_task(self,):
        
        #make transform tasl for input nodes, if needed
        for node in self.input_nodes:
            try: node.transform_task
            except NotFittedError:
                node._make_transform_task(node.transform_method)

                
        self.__fit_task = fit_task_factory(self.name + '__fit', self.estimator, [i.transform_task for i in self.input_nodes], self.y_loader)
        self.__estimator_path = self.fit_task().output().path
        return
    
    def _fit(self):        
                                
        #make fit task
        self._make_fit_task()
        #run task
        d6tflow.Workflow(self.fit_task).run()                
        return self
    
    def fit(self, X, y = None, **kwargs):
        raise NotImplementedError('To fit this estimator, create a DAG and call fit in this instance')
        
    def _infer(self, X = None, infer_method = None, **kwargs):        
        '''
        abstract inference interface, works for different methods, such as predict, predict proba...
        '''
        self._make_transform_task(transform_method = infer_method)                        
        #run task
        workflow = d6tflow.Workflow(self.transform_task)
        workflow.run()        
        self.__output_path = self.transform_task().output().path
        return workflow.outputLoad()
    
    def transform(self, X = None, **kwargs):
        return self._infer(X, infer_method = 'transform', **kwargs)
    
    def predict(self, X = None, **kwargs):
        return self._infer(X, infer_method = 'predict', **kwargs)
    
    def predict_proba(self, X = None, **kwargs):
        return self._infer(X, infer_method = 'predict_proba', **kwargs)
        

# ConcatenateNode

In [10]:
#export

def _concat(X):
    
    '''
    concatenate function, handles mixed sparse and dense
    '''

    if not isinstance(X, (tuple, list)):
        raise TypeError(f'X must be list or tuple, got {type(X)}')

    X = list(X)
    for i in range(len(X)):
        if len(X[i].shape) < 2:
            X[i] = X[i].reshape(-1,1)
    
    if any([sparse.issparse(x) for x in X]):
        X = sparse.hstack(X)
    else:                
        X = np.hstack(X)

    return X

class ConcatenateNode(NodeTransformer):
    '''
    transformer to concatenate (hstack) arrays in a tuple or list
    '''
    def __init__(self, input_nodes, name = None):        
        super().__init__(FunctionTransformer(_concat), input_nodes = input_nodes, name = name)
        return        

# CustomYNode

In [11]:
class CustomYNode(NodeTransformer):
    '''
    recieves X inputs and y inputs, that is, the learning task (y) can be defined as the output of
    some node, as well as the input values (X)
    '''
    pass

# Export

In [24]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted d6tflow-sklearn.ipynb.
Converted dag.ipynb.
Converted node.ipynb.
Converted utils.ipynb.
